# Handwritten numbers Recognition

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras 

## Load Data

Extraer datos. Las imágenes son de 28*28 píxeles, por lo tanto el csv tiene 784 columnas más 1 indicando la salida

In [2]:
def extract():
    df_train = pd.read_csv('data/mnist_train.csv', delimiter = ',').to_numpy()
    df_test = pd.read_csv('data/mnist_test.csv', delimiter = ',').to_numpy()
    df = pd.DataFrame(np.concatenate((df_train,df_test)))
    return df

Transformar output a array de 10 y separar de las inputs

In [3]:
def transform(df): 
    y_raw = df.iloc[:, 0].to_numpy()
    x = df.drop(df.columns[0], axis='columns').to_numpy()
    y = []
    for i in range(0,y_raw.shape[0]):
        row=np.zeros(10)
        row[y_raw[i]]=1
        y.append(row)
    y=np.array(y)
    y.reshape(y.shape[0],10)
    return x, y

Creamos los dataset de entrenamiento y test

In [4]:
df = extract()
x_train,y_train = transform(df.iloc[0:60000])
#x_val,y_val = transform(df.iloc[50000:60000])
x_test,y_test = transform(df.iloc[60000:70000])

In [5]:
x_train = x_train.reshape(60000,28,28,1)
x_test = x_test.reshape(9998,28,28,1)

## Creación de la Red Neuronal

En la red neuronal vamoas a implementar una arquitectura de (784,512,256,128,10), con funciones de activación ReLu en todas las capas menos en la última, en la cual utilizaremos la funcion softmax (https://www.quora.com/Why-is-it-better-to-use-Softmax-function-than-sigmoid-function).<br> Para cada capa se utilizará la funcion Dense, la cual tiene los siguientes parámetros por defecto:
- activation=None,
- use_bias=True,
- kernel_initializer='glorot_uniform',
- bias_initializer='zeros',
- kernel_regularizer=None,
- bias_regularizer=None,
- activity_regularizer=None,
- kernel_constraint=None,
- bias_constraint=None




In [6]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
# next weight/high size = (n+2p-f)/s +1 
# n = current weight/high
# p = pad
# s = stride
# f = filter dim

Using TensorFlow backend.


## Sin pooling

Instanciamos el modelo, 2 capas convolucionales de 64 y 32

In [7]:
model = Sequential()
#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)))#capa de 64 filtros de 3x3. dims 64*28*28*1
model.add(Conv2D(32, kernel_size=3, activation='relu'))#capa de 32 filtros de 3x3. dims 32*28*28*1
model.add(Flatten())# pasa los resultados de matriz a vector
model.add(Dense(10, activation='softmax'))#dicho vector se conecta a una capa para generar la salida

Entrenamiento, 3 epochs

In [8]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])#learning_rate=0.001
model.fit(x_train, y_train,validation_data=(x_test, y_test), batch_size=200, epochs=3)

Train on 60000 samples, validate on 9998 samples
Epoch 1/3
60000/60000 [==============================] - 22s 365us/step - loss: 0.4154 - accuracy: 0.9358 - val_loss: 0.0768 - val_accuracy: 0.9742
Epoch 2/3
60000/60000 [==============================] - 13s 209us/step - loss: 0.0572 - accuracy: 0.9828 - val_loss: 0.0745 - val_accuracy: 0.9780
Epoch 3/3
60000/60000 [==============================] - 13s 219us/step - loss: 0.0340 - accuracy: 0.9888 - val_loss: 0.0993 - val_accuracy: 0.9750


In [9]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test Acc: ', test_acc)

9998/9998 [==============================] - 2s 204us/step
Test Acc:  0.9749950170516968


### Con Maxpooling

Añadimos una capa maxpool: 2*2, stride=1, padding = valid

In [11]:
model = Sequential()

#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)))#capa de 64 filtros de 3x3. dims 64*28*28*1
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None))
model.add(Conv2D(32, kernel_size=3, activation='relu'))#capa de 32 filtros de 3x3. dims 32*28*28*1
model.add(Flatten())# pasa los resultados de matriz a vector
model.add(Dense(10, activation='softmax'))#dicho vector se conecta a una capa para generar la salida

Vamos a crear y entrenar el modelo, 3 epochs serán suficientes, el tiempo de entrenamiento se reduce un 50% aproximdamente.

In [12]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])#learning_rate=0.001
model.fit(x_train, y_train,validation_data=(x_test, y_test), batch_size=200, epochs=3)#sin pool: 3min ish, acc:0.982096

Train on 60000 samples, validate on 9998 samples
Epoch 1/3
60000/60000 [==============================] - 9s 153us/step - loss: 0.5430 - accuracy: 0.9179 - val_loss: 0.0822 - val_accuracy: 0.9747
Epoch 2/3
60000/60000 [==============================] - 8s 138us/step - loss: 0.0703 - accuracy: 0.9788 - val_loss: 0.0682 - val_accuracy: 0.9793
Epoch 3/3
60000/60000 [==============================] - 8s 137us/step - loss: 0.0480 - accuracy: 0.9858 - val_loss: 0.0719 - val_accuracy: 0.9810


Evaluamos el modelo, los resultados mejoran.

In [13]:
tes_loss, test_acc = model.evaluate(x_test, y_test)
print('Test Acc: ', test_acc)

9998/9998 [==============================] - 1s 108us/step
Test Acc:  0.9809961915016174


### Max Pooling 2

In [14]:
model = Sequential()

#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)))#capa de 64 filtros de 3x3. dims 64*28*28*1
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid', data_format=None))
model.add(Conv2D(32, kernel_size=3, activation='relu'))#capa de 32 filtros de 3x3. dims 32*28*28*1
model.add(Flatten())# pasa los resultados de matriz a vector
model.add(Dense(10, activation='softmax'))#dicho vector se conecta a una capa para generar la salida

Esta vez con stride = 2

In [15]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])#learning_rate=0.001
model.fit(x_train, y_train,validation_data=(x_test, y_test), batch_size=200, epochs=3)

Train on 60000 samples, validate on 9998 samples
Epoch 1/3
60000/60000 [==============================] - 8s 140us/step - loss: 0.5998 - accuracy: 0.9122 - val_loss: 0.0883 - val_accuracy: 0.9717
Epoch 2/3
60000/60000 [==============================] - 8s 138us/step - loss: 0.0714 - accuracy: 0.9787 - val_loss: 0.0650 - val_accuracy: 0.9799
Epoch 3/3
60000/60000 [==============================] - 9s 142us/step - loss: 0.0478 - accuracy: 0.9857 - val_loss: 0.0636 - val_accuracy: 0.9825


Mejoran los timepos, baja un poco el rendimiento

In [16]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test Acc: ', test_acc)

9998/9998 [==============================] - 1s 117us/step
Test Acc:  0.9824965000152588


### Common Structure

Usando padding=same

In [17]:
model = Sequential()

#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)))#capa de 64 filtros de 3x3. dims 64*28*28*1
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(Conv2D(32, kernel_size=3, activation='relu'))#capa de 32 filtros de 3x3. dims 32*28*28*1
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(Flatten())# pasa los resultados de matriz a vector
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='softmax'))#dicho vector se conecta a una capa para generar la salida

In [18]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])#learning_rate=0.001
model.fit(x_train, y_train,validation_data=(x_test, y_test), batch_size=200, epochs=3)

Train on 60000 samples, validate on 9998 samples
Epoch 1/3
60000/60000 [==============================] - 9s 153us/step - loss: 0.8068 - accuracy: 0.8912 - val_loss: 0.0902 - val_accuracy: 0.9723
Epoch 2/3
60000/60000 [==============================] - 9s 142us/step - loss: 0.0764 - accuracy: 0.9776 - val_loss: 0.0609 - val_accuracy: 0.9821
Epoch 3/3
60000/60000 [==============================] - 9s 142us/step - loss: 0.0512 - accuracy: 0.9842 - val_loss: 0.0603 - val_accuracy: 0.9815


In [19]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test Acc: ', test_acc)

9998/9998 [==============================] - 1s 98us/step
Test Acc:  0.981496274471283


### Common Structure 2

In [20]:
model = Sequential()
#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid', data_format=None))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid', data_format=None))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='softmax'))
#Compile and train
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])#learning_rate=0.001
model.fit(x_train, y_train,validation_data=(x_test, y_test), batch_size=200, epochs=3)
#Testing
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test Acc: ', test_acc)

Train on 60000 samples, validate on 9998 samples
Epoch 1/3
60000/60000 [==============================] - 9s 143us/step - loss: 1.0142 - accuracy: 0.8501 - val_loss: 0.1224 - val_accuracy: 0.9610
Epoch 2/3
60000/60000 [==============================] - 8s 141us/step - loss: 0.1006 - accuracy: 0.9689 - val_loss: 0.0814 - val_accuracy: 0.9726
Epoch 3/3
9998/9998 [==============================] - 1s 97us/step
Test Acc:  0.979395866394043
